# Demo For A New Prototype Of JuliaBUGS

We are exploring a lighter-weight compiler that does not rely on Symbolic Algebra System.

## What do we gain?
The new implementation is more modular and scalable. 

## Difference in Semantics
Symbolic-version of the compiler extends the semantic of the BUGS language in the following ways:
1. allowing using logical variable defined in the model specification almost anywhere in the model specification
2. allowing generic expressions (e.g., function calls) in indices or loop bounds
3. simplify model specification, provide the Symbolic algebra system

But the price we pay is that we need to unroll all the for loops and keep a symbolic expression for every variables. In the case where that are many variables, this can be slow. 
Aiming at provide a compiler that's more modular and more performant, we made a new prototype closer to the original BUGS implementations.

## Introducing passes
We adopt the idea of passes from classical compiler optimizations. 
Here, in a single `pass` of the program AST, we extract some specific information of the program. 
Passes should do simple things, so that them can be combined in different ways to achieve different targets.

## Passes we implemented
For now, we have implemented three passes:
1. collect variables: this pass enumerate all possible assignments by running the for loops

In [1]:
using Graphs, JuliaBUGS, Distributions
using JuliaBUGS: CompilerPass, CollectVariables, DependencyGraph, NodeFunctions, program!, @bugsast, eval, Var, logjoint

[ Info: Precompiling JuliaBUGS [d71a52e4-b6e4-48cf-ab6f-7c78c729c779]


In [2]:
model_def = @bugsast begin
    a ~ dnorm(0, 1)
    b ~ dnorm(0, a)
    for i in 1:N
        c[i] ~ dnorm(a, b)
    end
    g = b * 2 + a
    d[1:3] ~ dmnorm(e[1:3], f[1:3, 1:3])
end

data = Dict(:N => 3, :f => [1 0 0; 0 1 0; 0 0 1], :e => [1, 2, 3])
initializations = Dict(:a => 1, :b => 2, :c => [1, 2, 3], :d => [4, 5, 6])

Dict{Symbol, Any} with 4 entries:
  :a => 1
  :b => 2
  :d => [4, 5, 6]
  :c => [1, 2, 3]

In [3]:
vars, arrays_map, var_types = program!(CollectVariables(), model_def, data);

In [4]:
vars

Vars(c[3] => 5, d[1] => 7, d[2] => 8, d[:] => 9, d[3] => 10, a => 1, g => 6, c[1] => 3, b => 2, c[2] => 4)

In [5]:
arrays_map

Dict{Any, Any} with 2 entries:
  :d => [7, 8, 10]
  :c => [3, 4, 5]

In [6]:
var_types

Dict{Any, Any} with 10 entries:
  c[3] => :stochastic
  d[1] => :stochastic
  d[2] => :stochastic
  d[:] => :stochastic
  d[3] => :stochastic
  a    => :stochastic
  g    => :logical
  c[1] => :stochastic
  b    => :stochastic
  c[2] => :stochastic

2. construct a dependency graph: find the dependency relations between variables, this information can be used in Gibbs-type logdensity computation or to assist code generation in source-to-source transformation.

In [7]:
vars, dep_graph = program!(DependencyGraph(vars, arrays_map), model_def, data);

In [8]:
vars # we added a variable c[:] representing the array containing all the c variables

Vars(c[:] => 11, d[:] => 9, a => 1, b => 2, d[1] => 7, d[2] => 8, d[3] => 10, g => 6, c[2] => 4, c[3] => 5, c[1] => 3)

In [9]:
for e in edges(dep_graph)
    println(vars.id_var_map[src(e)], " -> ", vars.id_var_map[dst(e)])
end

a -> b
a -> c[1]
a -> c[2]
a -> c[3]
a -> g
b -> c[1]
b -> c[2]
b -> c[3]
b -> g
c[1] -> c[:]
c[2] -> c[:]
c[3] -> c[:]
d[:] -> d[1]
d[:] -> d[2]
d[:] -> d[3]


3. determine node functions: for a logical variable, the node function will evaluate to a value that has the same type of the variable; for a stochastic variable, the node function will evaluate to a distribution object that can be scored.

In [10]:
node_args, node_funcs = program!(NodeFunctions(), model_def, data, arrays_map);

In [11]:
node_args

Dict{Any, Any} with 7 entries:
  c[3] => Any[a, b]
  d[:] => Any[]
  a    => Any[]
  g    => Any[a, b]
  c[1] => Any[a, b]
  b    => Any[a]
  c[2] => Any[a, b]

In [13]:
for k in keys(node_funcs)
    println(k, ": ", node_args[k], " -> ", node_funcs[k])
end

c[3]: Any[a, b] -> #67
d[:]: Any[] -> #71
a: Any[] -> #63
g: Any[a, b] -> #69
c[1]: Any[a, b] -> #67
b: Any[a] -> #65
c[2]: Any[a, b] -> #67


In [17]:
@code_lowered node_funcs[Var(:c, [1])](1, 2)

CodeInfo(
1 ─ %1 = JuliaBUGS.dnorm(arg#292, arg#293)
└──      return %1
)

In [22]:
@code_lowered node_funcs[Var(:d, [1:3])]()

CodeInfo(
1 ─ %1 = JuliaBUGS.MvNormal([1.0, 2.0, 3.0], [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0])
└──      return %1
)

With the above intermediate result from the passes, evaluating the log joint probability is straightforward: traverse the nodes according to topological order and accumulate logdensity.

In [20]:
# process initializations
trace = Dict()
for var in keys(var_types)
    if var_types[var] == :stochastic
        value = JuliaBUGS.eval(var, initializations)
        isnothing(value) && (value = JuliaBUGS.eval(var, data))
        trace[var] = value
    end
end

# compute logjoint
sorted_node = filter(x -> vars.id_var_map[x] in keys(node_funcs), topological_sort_by_dfs(dep_graph))
logical_trace = Dict()
logdensity = 0.0
for s in sorted_node
    v = vars.id_var_map[s]
    arguments = []
    for arg in node_args[v]
        if arg in keys(trace)
            push!(arguments, trace[arg])
        else
            push!(arguments, logical_trace[arg])
        end
    end
    if var_types[v] == :logical
        logical_trace[v] = Base.invokelatest(node_funcs[v], arguments...)
    else
        logdensity += logpdf(Base.invokelatest(node_funcs[v], arguments...), trace[v])
    end
end
return logdensity

-27.311787494797468